In [1]:
using Revise

In [2]:
using Pkg
pkg"activate .."

  Activating environment at `~/.julia/dev/SymArrays/Project.toml`


In [3]:
#pkg"up"

In [4]:
using BenchmarkTools
using TupleTools
using MacroTools

In [5]:
using SymArrays
using SymArrays: SymIndexIter, ind2sub_symgrp
using Base: tail

┌ Info: Precompiling SymArrays [c2b21f84-4a8e-4c96-99a2-2b79df568f75]
└ @ Base loading.jl:1342


In [6]:
consume(it) = begin
    ii = 0
    for x in it
        ii += first(Tuple(x))
    end
    ii
end

consume (generic function with 1 method)

In [7]:
@btime consume(SI) setup=(SI=SymIndexIter(8,10)) seconds=1

  30.657 μs (0 allocations: 0 bytes)


48620

In [8]:
S = SymArray{(2,3,2),Float64}(9,9,6,6,6,4,4);
# S = SymArray{(3,),Float64}(9,9,9);

In [9]:
length(S)

25200

In [10]:
S.Nts

(9, 6, 4)

In [11]:
SymArrayIter2(S::SymArray) = (CartesianIndex(TupleTools.flatten(I)) for I in Base.Iterators.product(SymIndexIter.(symgrps(S),S.Nts)...))

SymArrayIter2 (generic function with 1 method)

In [12]:
struct SymArrayIter3{N,T}
    iters::T
    SymArrayIter3(S::SymArray{Nsyms,T,N}) where {Nsyms,T,N} = (iters = SymIndexIter.(symgrps(S),S.Nts); new{N,typeof(iters)}(iters))
end
Base.IteratorSize(::Type{<:SymArrayIter3}) = Base.HasLength()
Base.IteratorEltype(::Type{<:SymArrayIter3}) = Base.HasEltype()
Base.ndims(::SymArrayIter3{N,T}) where {N,T} = N
Base.eltype(::Type{SymArrayIter3{N,T}}) where {N,T} = NTuple{N,Int}
#Base.length(iter::SymArrayIter3{N,T}) where {N,T} = symarrlength(Nts,Nsyms)
Base.first(iter::SymArrayIter3) = CartesianIndex(TupleTools.flatten(first.(iter.iters)))
Base.last(iter::SymArrayIter3) = CartesianIndex(TupleTools.flatten(last.(iter.iters)))

In [13]:
function Base.iterate(iter::SymArrayIter3)
    Is = first.(iter.iters)
    CartesianIndex(TupleTools.flatten(Is)), Is
end
function Base.iterate(iter::SymArrayIter3, states)
    valid, Is = __inc3(iter.iters, states)
    valid || return nothing
    CartesianIndex(TupleTools.flatten(Is)), Is
end

function __inc3(iters::Tuple{T}, states::Tuple{<:Tuple}) where T
    v, I1 = SymArrays.__inc(first(iters),first(states))
    v, (I1,)
end
function __inc3(iters::Tuple, states::S)::Tuple{Bool,S} where S
    #println(states)
    valid, I1 = SymArrays.__inc(first(iters),first(states))
    #println(valid,I1)
    if valid
        valid, (I1, Base.tail(states)...)
    else
        I1 = first(first(iters))
        valid, Irest = __inc3(Base.tail(iters), Base.tail(states))
        valid, (I1, Irest...)
    end
end;

In [14]:
SymArrays.symarrlength((2,3,4),(3,-2,2))

120

In [15]:
struct SymArrayIter4{Nsyms,N}
    sizes::NTuple{N,Int}
    SymArrayIter4(A::SymArray{Nsyms,T,N}) where {Nsyms,T,N} = new{Nsyms,N}(A.size)
end

@generated function Base.first(iter::SymArrayIter4{Nsyms}) where Nsyms
    # have this be a generated function so the return value is 
    # simply the constant final tuple
    CartesianIndex(TupleTools.flatten(first.(SymIndexIter.(Nsyms,Nsyms))))
end

@inline function Base.iterate(iter::SymArrayIter4)
    cI = first(iter)
    cI, Tuple(cI)
end

function SymArrayIter4_iterate_code(Nsyms)
    newstate = Any[:(state[$i]) for i=1:sum(abs.(Nsyms))]
    code = quote end
    # global dimension index
    D = 0
    for Nsym in Nsyms
        for d = 1:abs(Nsym)
            D += 1
            maxv = d==abs(Nsym) ? :(iter.sizes[$D]) : (Nsym>0 ? :(state[$(D+1)]) : :(state[$(D+1)]-1))
            newstate[D] = :( state[$D] + 1 )
            push!(code.args, :( state[$D] < $maxv && (I=($(newstate...),); return (CartesianIndex(I),I)) ) )
            newstate[D] = Nsym>0 ? 1 : d
        end
    end
    push!(code.args, :(nothing) )
    code
end

function SymArrayIter4_iterate_code2(Nsyms,N)
    newstate = Any[:(state[$i]) for i=1:N]
    code = Expr(:block)
    currif = Expr(:if)
    push!(code.args,currif)
    # global dimension index
    D = 0
    for Nsym in Nsyms
        for d = 1:abs(Nsym)
            D += 1
            maxv = d==abs(Nsym) ? :(iter.sizes[$D]) : (Nsym>0 ? :(state[$(D+1)]) : :(state[$(D+1)]-1))
            newstate[D] = :( state[$D] + 1 )
            push!(currif.args, :( state[$D] < $maxv ))
            push!(currif.args, :( I = ($(newstate...),) ))
            newstate[D] = Nsym>0 ? 1 : d
            if D == N
                push!(currif.args, :( return nothing ))
            else
                push!(currif.args, Expr(:elseif))
                currif = currif.args[3]
            end
        end
    end
    push!(code.args, :( CartesianIndex(I), I ))
    code
end

@generated function Base.iterate(iter::SymArrayIter4{Nsyms,N}, state::NTuple{N,Int}) where {Nsyms,N}
    SymArrayIter4_iterate_code2(Nsyms,N)
end

In [16]:
MacroTools.prettify(SymArrayIter4_iterate_code2((2,-3,2),7))

quote
    if state[1] < state[2]
        I = (state[1] + 1, state[2], state[3], state[4], state[5], state[6], state[7])
    elseif state[2] < iter.sizes[2]
        I = (1, state[2] + 1, state[3], state[4], state[5], state[6], state[7])
    elseif state[3] < state[4] - 1
        I = (1, 1, state[3] + 1, state[4], state[5], state[6], state[7])
    elseif state[4] < state[5] - 1
        I = (1, 1, 1, state[4] + 1, state[5], state[6], state[7])
    elseif state[5] < iter.sizes[5]
        I = (1, 1, 1, 2, state[5] + 1, state[6], state[7])
    elseif state[6] < state[7]
        I = (1, 1, 1, 2, 3, state[6] + 1, state[7])
    elseif state[7] < iter.sizes[7]
        I = (1, 1, 1, 2, 3, 1, state[7] + 1)
    else
        return nothing
    end
    (CartesianIndex(I), I)
end

In [17]:
S = SymArray{(2,3,2),Float64}(9,9,6,6,6,12,12);

SA1 = SymArrays.SymArrayIter(S);
SA2 = SymArrayIter2(S);
SA3 = SymArrayIter3(S);
SA4 = SymArrayIter4(S);
for (I1,I2,I3,I4) in zip(SA1,SA2,SA3,SA4)
    I1 == I2 == I3 == I4 || error("I1: $I1, I2: $I2, I3: $I3, I4: $I4")
end
for iter in (SA1,SA2,SA3,SA4)
    @assert consume(iter) == consume(SA4)
    @assert count(I->true,iter) == count(I->true,SA4)
end

In [18]:
@btime consume($SA1)
@btime consume($SA2)
@btime consume($SA3)
@btime consume($SA4)

  87.887 μs (0 allocations: 0 bytes)
  263.285 μs (4 allocations: 96 bytes)
  190.269 μs (0 allocations: 0 bytes)
  84.958 μs (0 allocations: 0 bytes)


720720

In [19]:
@btime count(I->true,$SA1)
@btime count(I->true,$SA2)
@btime count(I->true,$SA3)
@btime count(I->true,$SA4)

  1.221 μs (0 allocations: 0 bytes)
  244.782 μs (4 allocations: 96 bytes)
  126.880 μs (0 allocations: 0 bytes)
  1.220 μs (0 allocations: 0 bytes)


196560

In [20]:
S = SymArray{(2,-3,2),Float64}(9,9,6,6,6,12,12);

SA1 = eachindex(S);
SA2 = SymArrayIter2(S);
SA3 = SymArrayIter3(S);
SA4 = SymArrayIter4(S);
for (I1,I2,I3,I4) in zip(SA1,SA2,SA3,SA4)
    I1 == I2 == I3 == I4 || error("I1: $I1, I2: $I2, I3: $I3, I4: $I4")
end
for iter in (SA1,SA2,SA3,SA4)
    @assert consume(iter) == consume(SA4)
    @assert count(I->true,iter) == count(I->true,SA4)
end

In [21]:
@btime consume($SA1)
@btime consume($SA2)
@btime consume($SA3)
@btime consume($SA4)

  29.925 μs (0 allocations: 0 bytes)
  95.901 μs (4 allocations: 96 bytes)
  56.551 μs (0 allocations: 0 bytes)
  29.924 μs (0 allocations: 0 bytes)


257400

In [22]:
@btime count(I->true,$SA1)
@btime count(I->true,$SA2)
@btime count(I->true,$SA3)
@btime count(I->true,$SA4)

  866.638 ns (0 allocations: 0 bytes)
  84.878 μs (4 allocations: 96 bytes)
  50.747 μs (0 allocations: 0 bytes)
  861.426 ns (0 allocations: 0 bytes)


70200

In [23]:
for n in (10,20)
    for d in 1:4
        for D = (-d,d)
            S = SymArray{(D,),Float64}(ntuple(i->n,d)...)
            print("n: $n, d: $d, count:   "); @btime count(I->true,eachindex($S))
            #print("n: $n, d: $d: consume: "); @btime consume(eachindex($S))
        end
    end
    println()
end

n: 10, d: 1, count:     1.815 ns (0 allocations: 0 bytes)
n: 10, d: 1, count:     1.815 ns (0 allocations: 0 bytes)
n: 10, d: 2, count:     61.581 ns (0 allocations: 0 bytes)
n: 10, d: 2, count:     56.470 ns (0 allocations: 0 bytes)
n: 10, d: 3, count:     20.159 ns (0 allocations: 0 bytes)
n: 10, d: 3, count:     21.117 ns (0 allocations: 0 bytes)
n: 10, d: 4, count:     73.747 ns (0 allocations: 0 bytes)
n: 10, d: 4, count:     131.347 ns (0 allocations: 0 bytes)

n: 20, d: 1, count:     1.815 ns (0 allocations: 0 bytes)
n: 20, d: 1, count:     2.168 ns (0 allocations: 0 bytes)
n: 20, d: 2, count:     268.915 ns (0 allocations: 0 bytes)
n: 20, d: 2, count:     373.688 ns (0 allocations: 0 bytes)
n: 20, d: 3, count:     41.174 ns (0 allocations: 0 bytes)
n: 20, d: 3, count:     44.589 ns (0 allocations: 0 bytes)
n: 20, d: 4, count:     337.922 ns (0 allocations: 0 bytes)
n: 20, d: 4, count:     458.872 ns (0 allocations: 0 bytes)



In [15]:
# struct TstIter{Nsym}
#     size::Int
#     "create an iterator that gives i1<=i2<=i3 etc for one index group"
#     TstIter(Nsym,size) = new{Nsym}(size)
# end
# Base.IteratorSize(::Type{<:TstIter}) = Base.HasLength()
# Base.IteratorEltype(::Type{<:TstIter}) = Base.HasEltype()
# Base.ndims(::TstIter{Nsym}) where Nsym = Nsym
# Base.eltype(::Type{TstIter{Nsym}}) where Nsym = NTuple{Nsym,Int}
# Base.length(iter::TstIter{Nsym}) where Nsym = symgrp_size(iter.size,Nsym)
# Base.first(iter::TstIter{Nsym}) where Nsym = Nsym > 0 ? ntuple(one,Val(Nsym)) : ntuple(i->i,Val(-Nsym))
# Base.last(iter::TstIter{Nsym}) where Nsym = Nsym>0 ? ntuple(i->iter.size,Val(Nsym)) : ntuple(i->iter.size+Nsym+i #= Nsym < 0 here =#,Val(-Nsym))

# @inline function Base.iterate(iter::TstIter)
#     I = first(iter)
#     I, I
# end
# @inline function Base.iterate(iter::TstIter, state)
#     valid, I = __tstinc(iter, state)
#     ifelse(valid, (I, I), nothing)
# end

# function __tstinc(iter::TstIter{Nsym}, state::NTuple{N,Int}) where {Nsym,N}
#     if state[1] + (Nsym<0) < state[2]
#         return true, (state[1]+1, tail(state)...)
#     end
#     valid, I = __tstinc(iter, tail(state))
#     # if Nsym<0, lowest possible value for nth index is n
#     In = ifelse(Nsym>0, 1, 1 - Nsym - N)
#     return valid, (In, I...)
# end
# function __tstinc(iter::TstIter, state::Tuple{Int})
#     # last index can go until iter.size independent of (anti-)symmetry
#     state[1] < iter.size, (state[1]+1,)
# end

__tstinc (generic function with 2 methods)

In [16]:
symgrp_size(10,-4)

210

In [18]:
fbench = x -> count(i->true,x)
#fbench = consume
for n in (10,20) #,30,40,50)
    for d in 2:4
        for (iter,name) in ((SymIndexIter(d,n),"PkgS"),
                            (SymIndexIter(-d,n),"PkgA"),
                            (TstIter(d,n),"TstS"),
                            (TstIter(-d,n),"TstA"))
            @assert count(i->true,iter) == length(iter)
            print("n: $n, d: $d, $name: "); @btime $fbench($iter)
        end
        println()
    end
    println()
end

n: 10, d: 2, PkgS:   2.096 ns (0 allocations: 0 bytes)
n: 10, d: 2, PkgA:   2.026 ns (0 allocations: 0 bytes)
n: 10, d: 2, TstS:   1.756 ns (0 allocations: 0 bytes)
n: 10, d: 2, TstA:   2.171 ns (0 allocations: 0 bytes)

n: 10, d: 3, PkgS:   19.719 ns (0 allocations: 0 bytes)
n: 10, d: 3, PkgA:   18.790 ns (0 allocations: 0 bytes)
n: 10, d: 3, TstS:   19.717 ns (0 allocations: 0 bytes)
n: 10, d: 3, TstA:   18.779 ns (0 allocations: 0 bytes)

n: 10, d: 4, PkgS:   126.891 ns (0 allocations: 0 bytes)
n: 10, d: 4, PkgA:   71.109 ns (0 allocations: 0 bytes)
n: 10, d: 4, TstS:   126.287 ns (0 allocations: 0 bytes)
n: 10, d: 4, TstA:   71.221 ns (0 allocations: 0 bytes)


n: 20, d: 2, PkgS:   2.025 ns (0 allocations: 0 bytes)
n: 20, d: 2, PkgA:   1.693 ns (0 allocations: 0 bytes)
n: 20, d: 2, TstS:   1.698 ns (0 allocations: 0 bytes)
n: 20, d: 2, TstA:   2.027 ns (0 allocations: 0 bytes)

n: 20, d: 3, PkgS:   43.086 ns (0 allocations: 0 bytes)
n: 20, d: 3, PkgA:   40.431 ns (0 allocations: 0 

In [29]:
SI = SymIndexIter(-5,10)
length(SI), ind2sub_symgrp(SI,1)

(252, (1, 2, 3, 4, 5))

In [30]:
qu = quote
    a = 3b
end
cc = Expr(:if)
push!(qu.args,cc)
push!(cc.args,:(x<3))
push!(cc.args,:(2*a; a^2))
push!(cc.args,Expr(:elseif))
ce = cc.args[3]
push!(ce.args,:y)
push!(ce.args,:b)
push!(ce.args,:c)
qu

quote
    #= In[30]:2 =#
    a = 3b
    if x < 3
        2a
        #= In[30]:7 =#
        a ^ 2
    elseif y
        b
    else
        c
    end
end

In [31]:
using MacroTools

In [56]:
struct TstIterG{M} n::Int end

function Base.iterate(iter::TstIterG{M}) where M
    I = M > 0 ? ntuple(i->1,Val(M)) : ntuple(i->i,Val(-M))
    I, I
end
# function iterate_code(M)
#     Is = Any[Symbol(:i,i) for i=1:M]
#     code = quote
#         ($(Is...),) = state
#     end
#     Iexpr = Expr(:(=),:(valid,I),Expr(:if))
#     currif = Iexpr.args[2]
#     push!(code.args,Iexpr)
#     for d = 1:M
#         maxv = d<M ? Is[d+1] : :(iter.n)
#         push!(currif.args, :( $(Is[d]) < $maxv ) )
#         Is[d] = :( $(Symbol(:i,d)) + 1 )
#         push!(currif.args, :( (true, ($(Is...),)) ))
#         Is[d] = 1
#         push!(currif.args, d<M ? Expr(:elseif) : :((false,state)))
#         currif = currif.args[3]
#     end
#     push!(code.args, :( valid ? (I,I) : nothing) )
#     code
# end
function iterate_code(M)
    Is = Any[:(state[$i]) for i=1:abs(M)]
    code = quote end
    for d = 1:abs(M)
        maxv = d==abs(M) ? :(iter.n) : (M>0 ? Is[d+1] : :($(Is[d+1])-1))
        Is[d] = :( state[$d] + 1 )
        push!(code.args, :( state[$d] < $maxv && (I=($(Is...),); return (I,I)) ) )
        Is[d] = M>0 ? 1 : d
    end
    push!(code.args, :(nothing) )
    code
end

@generated function Base.iterate(iter::TstIterG{M}, state::NTuple{N,Int}) where {M,N}
    @assert N == abs(M)
    iterate_code(M)
end

In [57]:
MacroTools.prettify(iterate_code(-5))

quote
    state[1] < state[2] - 1 && begin
            I = (state[1] + 1, state[2], state[3], state[4], state[5])
            return (I, I)
        end
    state[2] < state[3] - 1 && begin
            I = (1, state[2] + 1, state[3], state[4], state[5])
            return (I, I)
        end
    state[3] < state[4] - 1 && begin
            I = (1, 2, state[3] + 1, state[4], state[5])
            return (I, I)
        end
    state[4] < state[5] - 1 && begin
            I = (1, 2, 3, state[4] + 1, state[5])
            return (I, I)
        end
    state[5] < iter.n && begin
            I = (1, 2, 3, 4, state[5] + 1)
            return (I, I)
        end
    nothing
end

In [61]:
fbench = x -> count(i->true,x)
#fbench = consume
for n in (10,20) #,30,40,50)
    for d in 2:4
        SI = SymIndexIter(d,n)
        AI = SymIndexIter(-d,n)
        SIg = TstIterG{d}(n)
        AIg = TstIterG{-d}(n)
        @assert fbench(SI)==fbench(SIg)
        @assert fbench(AI)==fbench(AIg)
        print("n: $n, d: $d, SI:  "); @btime $fbench($SI)
        print("n: $n, d: $d, SIg: "); @btime $fbench($SIg)
        print("n: $n, d: $d, AI:  "); @btime $fbench($AI)
        print("n: $n, d: $d, AIg: "); @btime $fbench($AIg)
        println()
    end
end


n: 10, d: 2, SI:    1.692 ns (0 allocations: 0 bytes)
n: 10, d: 2, SIg:   1.690 ns (0 allocations: 0 bytes)
n: 10, d: 2, AI:    1.695 ns (0 allocations: 0 bytes)
n: 10, d: 2, AIg:   1.691 ns (0 allocations: 0 bytes)

n: 10, d: 3, SI:    22.032 ns (0 allocations: 0 bytes)
n: 10, d: 3, SIg:   27.699 ns (0 allocations: 0 bytes)
n: 10, d: 3, AI:    18.691 ns (0 allocations: 0 bytes)
n: 10, d: 3, AIg:   19.358 ns (0 allocations: 0 bytes)

n: 10, d: 4, SI:    130.031 ns (0 allocations: 0 bytes)
n: 10, d: 4, SIg:   129.341 ns (0 allocations: 0 bytes)
n: 10, d: 4, AI:    78.528 ns (0 allocations: 0 bytes)
n: 10, d: 4, AIg:   78.560 ns (0 allocations: 0 bytes)

n: 20, d: 2, SI:    2.090 ns (0 allocations: 0 bytes)
n: 20, d: 2, SIg:   2.023 ns (0 allocations: 0 bytes)
n: 20, d: 2, AI:    2.021 ns (0 allocations: 0 bytes)
n: 20, d: 2, AIg:   2.020 ns (0 allocations: 0 bytes)

n: 20, d: 3, SI:    38.699 ns (0 allocations: 0 bytes)
n: 20, d: 3, SIg:   38.692 ns (0 allocations: 0 bytes)
n: 20, d: 3

In [40]:
@benchmark consume(SI) setup=(SI=TstIter(3,50)) seconds=1

BechmarkTools.Trial: 10000 samples with 950 evaluations.
 Range (min … max):   95.438 ns … 134.545 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      98.429 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   102.057 ns ±   4.958 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂      █    ▁▃▂▅     ▁ ▇       ▂▁▄                            ▂
  █▁▁▁▁▁▁█▅▃▃▄█████▃▁▁▁█▆█▅▅▃▃▄▃████▅▆▅▅▆▆▆█▇▇▆▆▆▆▆▆▆▆▆▆▆▇▇▆▆▆▇ █
  95.4 ns       Histogram: log(frequency) by time        121 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [183]:
println(@btime consume(SI) setup=(SI=TstIter(2,50)) seconds=1)

  1.676 μs (0 allocations: 0 bytes)
1275


In [42]:
println(@btime consume(SI) setup=(SI=SymArrays.SymIndexIter(2,50)) seconds=1)
println(@btime consume(SI) setup=(SI=TstIter(2,50)) seconds=1)

  1.382 μs (0 allocations: 0 bytes)
1275
  1.696 ns (0 allocations: 0 bytes)
1275


In [166]:
println(@btime consume(SI) setup=(SI=SymArrays.SymIndexIter(3,50)) seconds=1)
println(@btime consume(SI) setup=(SI=TstIter(3,50)) seconds=1)

  1.537 μs (0 allocations: 0 bytes)
22100
  94.773 ns (0 allocations: 0 bytes)
22100


In [43]:
SI = SymArrays.SymIndexIter(3,5)
println(first(SI))
println(last(SI))
println(collect(SI))

(1, 1, 1)
(5, 5, 5)
[(1, 1, 1), (1, 1, 2), (1, 2, 2), (2, 2, 2), (1, 1, 3), (1, 2, 3), (2, 2, 3), (1, 3, 3), (2, 3, 3), (3, 3, 3), (1, 1, 4), (1, 2, 4), (2, 2, 4), (1, 3, 4), (2, 3, 4), (3, 3, 4), (1, 4, 4), (2, 4, 4), (3, 4, 4), (4, 4, 4), (1, 1, 5), (1, 2, 5), (2, 2, 5), (1, 3, 5), (2, 3, 5), (3, 3, 5), (1, 4, 5), (2, 4, 5), (3, 4, 5), (4, 4, 5), (1, 5, 5), (2, 5, 5), (3, 5, 5), (4, 5, 5), (5, 5, 5)]


In [37]:
SI = SymArrays.SymIndexIter(3,5)
println(first(SI))
println(last(SI))
println(collect(SI))

(1, 1, 1)
(5, 5, 5)
[(1, 1, 1), (1, 1, 2), (1, 2, 2), (2, 2, 2), (1, 1, 3), (1, 2, 3), (2, 2, 3), (1, 3, 3), (2, 3, 3), (3, 3, 3), (1, 1, 4), (1, 2, 4), (2, 2, 4), (1, 3, 4), (2, 3, 4), (3, 3, 4), (1, 4, 4), (2, 4, 4), (3, 4, 4), (4, 4, 4), (1, 1, 5), (1, 2, 5), (2, 2, 5), (1, 3, 5), (2, 3, 5), (3, 3, 5), (1, 4, 5), (2, 4, 5), (3, 4, 5), (4, 4, 5), (1, 5, 5), (2, 5, 5), (3, 5, 5), (4, 5, 5), (5, 5, 5)]


In [153]:
using SymArrays: symgrp_sortedsub2ind, asymgrp_sortedsub2ind

In [154]:
function _sub2grp_code(Nsyms)
    permsigns = []
    result = []
    ii::Int = 0
    code = quote end
    for (isym,Nsym) in enumerate(Nsyms)
        Isrtd = Symbol(:Isrtd,isym)
        Ilocs = [ :(I[$(ii+=1)]) for _=1:abs(Nsym) ]
        Iloc = :( ($(Ilocs...),) )
        #println(Iloc)
        if Nsym > 0
            push!(code.args, :( $Isrtd = TupleTools.sort($Iloc) ))
            push!(permsigns, 1)
            push!(result, :( symgrp_sortedsub2ind($Isrtd...) ))
        else
            psym = Symbol(:p,isym)
            push!(code.args, :( ($psym, $Isrtd) = sort_track_parity($Iloc)) )
            push!(permsigns, psym)
            push!(result, :( asymgrp_sortedsub2ind($Isrtd...) ))
        end
    end
    push!(code.args, :( *($(permsigns...)), ($(result...),) ))
    code
end
@generated _sub2grp(A::SymArray{Nsyms,T,N}, I::Vararg{Int,N}) where {Nsyms,T,N} = _sub2grp_code(Nsyms)

_sub2grp_code((5,-2,3,-3))

quote
    #= In[154]:5 =#
    Isrtd1 = TupleTools.sort((I[1], I[2], I[3], I[4], I[5]))
    (p2, Isrtd2) = sort_track_parity((I[6], I[7]))
    Isrtd3 = TupleTools.sort((I[8], I[9], I[10]))
    (p4, Isrtd4) = sort_track_parity((I[11], I[12], I[13]))
    (1 * p2 * 1 * p4, (symgrp_sortedsub2ind(Isrtd1...), asymgrp_sortedsub2ind(Isrtd2...), symgrp_sortedsub2ind(Isrtd3...), asymgrp_sortedsub2ind(Isrtd4...)))
end

In [156]:
A = SymArray{(2,-3,3),Float64}(2,2,4,4,4,3,3,3)

2×2×4×4×4×3×3×3 SymArray{(2, -3, 3), Float64, 8, 3, Array{Float64, 3}}

In [163]:
checkbounds(rand(4,4),1,5)

LoadError: BoundsError: attempt to access 4×4 Matrix{Float64} at index [1, 5]

In [161]:
@boundscheck false

false

In [160]:
_sub2grp(A,2,1,3,1,4,4,2,1)

(-1, (2, 3, 12))